用几个常用的机器学习，对数据进行训练，进行预测，并对结果进行说明，不涉及说明机器学习算法原理。

In [ ]:
import akshare as ak
import pandas as pd
from __future__ import print_function
import datetime 
import numpy as np
import sklearn
import pandas_datareader.data as web
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import confusion_matrix 
from sklearn.svm import LinearSVC, SVC

In [2]:
## 1. 获取行情的例子
index_zh_a_hist_df = ak.index_zh_a_hist(symbol="000001", period="daily", start_date="20220101", end_date='20231231')
index_zh_a_hist_df.set_index(pd.to_datetime(index_zh_a_hist_df["日期"],format='%Y-%m-%d'),inplace=True)
print(index_zh_a_hist_df[0:3])

                    日期       开盘       收盘       最高       最低        成交量  \
日期                                                                      
2022-01-04  2022-01-04  3649.15  3632.33  3651.89  3610.09  405027768   
2022-01-05  2022-01-05  3628.26  3595.18  3628.26  3583.47  423902029   
2022-01-06  2022-01-06  3581.22  3586.08  3594.49  3559.88  371540544   

                     成交额    振幅   涨跌幅    涨跌额   换手率  
日期                                                 
2022-01-04  5.102511e+11  1.15 -0.20  -7.45  0.90  
2022-01-05  5.389636e+11  1.23 -1.02 -37.15  0.94  
2022-01-06  4.742843e+11  0.96 -0.25  -9.10  0.82  


In [3]:
# 创建数据，使用收盘价的涨跌幅作为元数据
# 这里使用的方法比较偏python，和直觉的思维有差异
def create_lagged_series(sym, startdate, enddate, lags=5): 
    # Obtain stock information 
    ts = ak.index_zh_a_hist(symbol=sym, period="daily", start_date=startdate, end_date=enddate)
    ts.set_index(pd.to_datetime(ts["日期"],format='%Y-%m-%d'),inplace=True)
    # Create the new lagged DataFrame
    tslag = pd.DataFrame(index=ts.index)
    tslag["Today"] = ts["收盘"]
    tslag["Volume"] = ts["成交量"]
    # Create the shifted lag series of prior trading period close values
    for i in range(0, lags):
        tslag["Lag%s" % str(i+1)] = ts["收盘"].shift(i+1)
    # Create the returns DataFrame
    tsret = pd.DataFrame(index=tslag.index) 
    tsret["Volume"] = tslag["Volume"]
    tsret["Today"] = tslag["Today"].pct_change()*100.0
    # If any of the values of percentage returns equal zero, set them to # a small number (stops issues with QDA model in Scikit-Learn)
    # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据
    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001): 
            tsret["Today"][i] = 0.0001
    # Create the lagged percentage returns columns
    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tslag["Lag%s" % str(i+1)].pct_change()*100.0
    # Create the "Direction" column (+1 or -1) indicating an up/down day
    # sign()是Python的Numpy中的取数字符号（数字前的正负号）的函数
    tsret["Direction"] = np.sign(tsret["Today"]) 
    startdatepd = pd.to_datetime(startdate,format='%Y%m%d')
    tsret = tsret[tsret.index >= startdatepd]
    
    return tsret

In [6]:

# Create a lagged series
lags_val = 2
snpret = create_lagged_series("000001", "20200101", "20231231", lags=lags_val)
print("----------->")
## 打印获取的数据集合
print(snpret)
# 过去2天的涨跌幅和当日的涨跌结果作为训练集合的输入和输出
X = snpret[["Lag1","Lag2"]]
y = snpret["Direction"]
# 前3年的数据作为训练集，对最近一年进行预测校验
start_test = datetime.datetime(2023,1,1)
X_train = X[X.index < start_test][lags_val + 1:]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test][lags_val + 1:]
y_test = y[y.index >= start_test]

print("Hit Rates/Confusion Matrices:\n") 
# Hit Rates 预测正确的比例
# Confusion Matrices
## [U1 U0]
## [D0 D1]
# U0 代表正确预测上涨，U0错误预测上涨；
# D0 错误预测下跌，D1 正确预测下跌
# 训练模型，作为一个列表，包含常用的模型
models = [("LR", LogisticRegression()),
    ("LDA", LDA()),("QDA", QDA()),("LSVC", LinearSVC()),
    ("RSVM", SVC(
    C=1000000.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None,
      shrinking=True, tol=0.001, verbose=False)
    ),
    ("RF", RandomForestClassifier(
      n_estimators=1000, criterion='gini',
      max_depth=None, min_samples_split=2,
    min_samples_leaf=1, 
                bootstrap=True, oob_score=False, n_jobs=1,
                random_state=None, verbose=0)
    )]
# Iterate through the models
for m in models:
    # Train each of the models on the training set
    m[1].fit(X_train, y_train)
 
    # Make an array of predictions on the test set
    pred = m[1].predict(X_test)
    # Output the hit-rate and the confusion matrix for each model
    print("%s:\n%0.3f" % (m[0], m[1].score(X_test, y_test))) 
    print("%s\n" % confusion_matrix(pred, y_test))

----------->
               Volume     Today      Lag1      Lag2  Direction
日期                                                            
2020-01-02  292470208       NaN       NaN       NaN        NaN
2020-01-03  261496668 -0.045702       NaN       NaN       -1.0
2020-01-06  312575841 -0.012322 -0.045702       NaN       -1.0
2020-01-07  276583112  0.693712 -0.012322 -0.045702        1.0
2020-01-08  297872553 -1.221013  0.693712 -0.012322       -1.0
...               ...       ...       ...       ...        ...
2023-12-25  229814178  0.138261 -0.134649  0.571998        1.0
2023-12-26  228140855 -0.682813  0.138261 -0.134649       -1.0
2023-12-27  247900882  0.542623 -0.682813  0.138261        1.0
2023-12-28  339213116  1.375484  0.542623 -0.682813        1.0
2023-12-29  290672687  0.684672  1.375484  0.542623        1.0

[970 rows x 5 columns]
Hit Rates/Confusion Matrices:

LR:
0.483
[[ 13  14]
 [111 104]]

LDA:
0.483
[[ 13  14]
 [111 104]]

QDA:
0.459
[[101 108]
 [ 23  10]]

LSVC:
0.4

/Users/neil/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RF:
0.455
[[63 71]
 [61 47]]



预测结果在0.455-0.483， 结果越接近0.5，说明越难以预测，接近随机扔硬币。